### OTHMANE NABGOURI


In [263]:
import numpy as np
import pandas as pd
from tabulate import tabulate

In [264]:
header_list = ["Classe", "X", "Y"]
data_train=pd.read_csv('data_tp1_app.txt', sep = ' ' ,names=header_list)
data_test=pd.read_csv('data_tp1_dec.txt', sep = ' ' ,names=header_list)
data_train2=pd.read_csv('data_tp2_app.txt', sep = ' ' ,names=header_list)
data_test2=pd.read_csv('data_tp2_dec.txt', sep = ' ' ,names=header_list)
data_train3=pd.read_csv('data_tp3_app.txt', sep = ' ' ,names=header_list)
data_test3=pd.read_csv('data_tp3_dec.txt', sep = ' ' ,names=header_list)




On fait ici aucune supposition sur la distribution des données. On considère juste que chaque classe a une densité de probabilité que respecte les échantillons de la vérité terrain , a chaque echantillon est attribué le noyau : 

$f(x) = \frac{1}{n} \sum_{i} \phi(x-x_{i})$

### 1-Création des noyaux

### Noyau uniform

On est dans deux dimentions , donc on va créer une densité de probabilité uniform de taille h*h , si la donnée a predire se trouve dans cette densité de probabilité , alors on retourne 1/h^^2 , sinon 0.
avec h : bandwith  


In [265]:
def noyau_uniform(x,y,h) :
    if (np.abs(x[0]-y[0]) <= h**2) and (np.abs(x[1]-y[1]) <= h**2) :
        return 1/(h**2)
    else :
        return 0

### Noyau gaussien

Il faut implémenter maintenant le Kernel suivant : $\phi(x,x_{i}) = \frac{1}{2\pi\sigma^{2}} e^{\frac{-||x-x_{i}||^{2}}{2\sigma^{2}}}$

In [267]:
def noyau_gaussien(x,y,h) :
    return np.exp(-(np.sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2))**2/(2*(h**2)))

### 2-Appliquer cette fenetre sur les données

In [268]:
###cette premiere fonction permet d'appliquer le noyau pour chaque donne de test avec tout les donnéés d'entrainement
### inconvenient : complexité proportionnel a N , c'est l'inconvenient de tout les methodes non parametriques
def apply_noyau(noyau,h,train,test):
    X=train.shape[0]
    Y=test.shape[0]
    Parzen = np.zeros((Y,X))
    for i in range(Y):
        for j in range(X):
            Parzen[i,j] = noyau(train.iloc[j,[1,2]],test.iloc[i,[1,2]],h)
    return Parzen
    

In [269]:
### Cette fonction permet de sommer pour chaque point de test , tout les valeurs non null du noyau , pour chaque classe !
### On aura a la fin une matrice dont chaque element de cordonnes(i,j) est la densité que l'element de test i appartient a la
### classe j
def density(Parzen,train,test):
    #Parzen = apply_noyau(noyau,h)
    X=train.shape[0]
    Y=test.shape[0]
    NOYAU = np.zeros((Y,5))
    for i in range(Y):
        k=0
        list_ctr=[]
        ctr=0
        for j in range(int(X/5),int((X/5))*6,int(X/5)):
            for t in range(k,j):
                    ctr+=Parzen[i,t]
            list_ctr.append(ctr)
            k=j
            ctr=0
        NOYAU[i]=np.array(list_ctr)
    return NOYAU
    


In [270]:
def get_closest(NOYAU,test):
    "indice des deux les plus proches"
    X,Y=test.shape
    indice_deux_proches= np.zeros((X,2))
    for k in range(X):
        indice_deux_proches[k]= pd.Series(NOYAU [k,:]).nlargest(2).index
    return  indice_deux_proches
    

In [271]:
def confusion_matrix(actual, predicted):
    Lenghtt = len(actual)
    top1=0
    top2=0
    # extraire les differents classes
    classes = np.unique(actual)

    matrix = np.zeros((len(classes), len(classes)))

    for i in range(len(classes)):
        for j in range(len(classes)):
            matrix[i, j] = np.sum((actual == classes[i]) & (predicted[:,0] == classes[j]))
    for k in range(Lenghtt):
        if actual[k] == predicted[k,0]:
            top1+=1
            top2+=1
        elif actual[k] == predicted[k,1]:
            top2+=1
            
    

    return matrix,top1/Lenghtt,top2/Lenghtt


### 3-Test 

### Noyau uniform

In [272]:
h=2

In [273]:
print('fenetre de parzen en utilisant les données de TP1')
Parzen=apply_noyau(noyau_uniform,h,data_train,data_test)
NOYAU=density(Parzen,data_train,data_test)
indice_deux_proches=get_closest(NOYAU,data_test)
Matrix,top1,top2=confusion_matrix(data_test.Classe,indice_deux_proches+1)
Matrix,top1,top2

fenetre de parzen en utilisant les données de TP1


(array([[ 99.,   0.,   0.,   0.,   1.],
        [  0., 100.,   0.,   0.,   0.],
        [  0.,   0., 100.,   0.,   0.],
        [  0.,   0.,   0., 100.,   0.],
        [  1.,   0.,   0.,   0.,  99.]]),
 0.996,
 0.998)

In [154]:
print('fenetre de parzen en utilisant les données de TP2')
Parzen=apply_noyau(noyau_uniform,h,data_train2,data_test2)
NOYAU=density(Parzen,data_train2,data_test2)
indice_deux_proches=get_closest(NOYAU,data_test2)
Matrix,top1,top2=confusion_matrix(data_test2.Classe,indice_deux_proches+1)
Matrix,top1,top2

fenetre de parzen en utilisant les données de TP2


(array([[100.,   0.,   0.,   0.,   0.],
        [  0.,  98.,   2.,   0.,   0.],
        [  0.,   8.,  81.,   0.,  11.],
        [  0.,   1.,   4.,  95.,   0.],
        [  0.,   0.,   1.,   0.,  99.]]),
 0.946,
 0.992)

In [153]:
print('fenetre de parzen en utilisant les données de TP3')
Parzen=apply_noyau(noyau_uniform,h,data_train3,data_test3)
NOYAU=density(Parzen,data_train3,data_test3)
indice_deux_proches=get_closest(NOYAU,data_test3)
Matrix,top1,top2=confusion_matrix(data_test3.Classe,indice_deux_proches+1)
Matrix,top1,top2

fenetre de parzen en utilisant les données de TP3


(array([[31., 15., 21., 15., 18.],
        [ 9., 79.,  7.,  0.,  5.],
        [13.,  3., 81.,  3.,  0.],
        [ 4.,  0.,  0., 81., 15.],
        [ 5., 10.,  0.,  0., 85.]]),
 0.714,
 0.898)

### Noyau Gaussien

In [97]:
h=2

In [155]:
print('fenetre de parzen en utilisant les données de TP1')
Parzen=apply_noyau(noyau_gaussien,h,data_train,data_test)
NOYAU=density(Parzen,data_train,data_test)
indice_deux_proches=get_closest(NOYAU,data_test)
Matrix,top1,top2=confusion_matrix(data_test.Classe,indice_deux_proches+1)
Matrix,top1,top2

fenetre de parzen en utilisant les données de TP1


(array([[100.,   0.,   0.,   0.,   0.],
        [  0., 100.,   0.,   0.,   0.],
        [  0.,   0.,  99.,   0.,   1.],
        [  0.,   0.,   0.,  99.,   1.],
        [  0.,   1.,   0.,   0.,  99.]]),
 0.994,
 1.0)

In [156]:
print('fenetre de parzen en utilisant les données de TP2')
Parzen2=apply_noyau(noyau_gaussien,h,data_train2,data_test2)
NOYAU2=density(Parzen2,data_train2,data_test2)
indice_deux_proches2=get_closest(NOYAU2,data_test2)
Matrix,top1,top2=confusion_matrix(data_test2.Classe,indice_deux_proches2+1)
Matrix,top1,top2

fenetre de parzen en utilisant les données de TP2


(array([[100.,   0.,   0.,   0.,   0.],
        [  0.,  98.,   2.,   0.,   0.],
        [  0.,   8.,  82.,   0.,  10.],
        [  0.,   1.,   4.,  95.,   0.],
        [  0.,   0.,   1.,   0.,  99.]]),
 0.948,
 0.992)

In [157]:
print('fenetre de parzen en utilisant les données de TP3')
Parzen=apply_noyau(noyau_gaussien,h,data_train3,data_test3)
NOYAU=density(Parzen,data_train3,data_test3)
indice_deux_proches=get_closest(NOYAU,data_test3)
Matrix,top1,top2=confusion_matrix(data_test3.Classe,indice_deux_proches+1)
Matrix,top1,top2

fenetre de parzen en utilisant les données de TP3


(array([[32., 15., 21., 16., 16.],
        [12., 78.,  6.,  0.,  4.],
        [14.,  3., 79.,  4.,  0.],
        [ 6.,  0.,  0., 87.,  7.],
        [ 6.,  5.,  0.,  0., 89.]]),
 0.73,
 0.9)

### Conclusion

In [158]:
table = [['Source des données/performance', 'TOP1 noyau_uniform', 'TOP2 noyau_uniform','TOP1 noyau_guassien','TOP2 noyau_guassien'], 
         ['TP1', '0.992', '1.0',' 0.994','1.0'], 
         ['TP2', '0.946', '0.992','0.948','0.992'], 
         ['TP3', '0.714', '0.898','0.73','0.9']]
print(tabulate(table))

------------------------------  ------------------  ------------------  -------------------  -------------------
Source des données/performance  TOP1 noyau_uniform  TOP2 noyau_uniform  TOP1 noyau_guassien  TOP2 noyau_guassien
TP1                             0.992               1.0                 0.994                1.0
TP2                             0.946               0.992               0.948                0.992
TP3                             0.714               0.898               0.73                 0.9
------------------------------  ------------------  ------------------  -------------------  -------------------


On remarque que les résultats obtenus avec le noyau gaussien sont meilleur par rapport au résultat obtenus par le noyau uniform , donc la distribution des données sont sous forme plus gaussienne que uniform

### 4-Cross validation

Pour optimiser la valeur de h , on va utiliser uniquement les données d'apprentissage car les données de test sont déstinés uniquement pour le test , et on a pas le droit d'ajuster notre parametre h en les utilisant . Comme ça , on aura un modéle qui est adabté au probleme et non au jeu de données.

k=5 ,et à chaque fois on aura 20% des données comme partie test , et 80% comme partie entrainement

In [210]:
def cross_validation(train):
    #On divise en premier temps le jeu de donnée en 5 classes :
    c1 = train[0:100]  
    c2 = train[100:200]
    c3 = train[200:300]
    c4 = train[300:400]
    c5= train[400:500]
    train1= pd.concat([c1.iloc[:80],c2.iloc[:80],c3.iloc[:80],c4.iloc[:80],c5.iloc[:80]],ignore_index=True )
    test1 = pd.concat([c1.iloc[80:100],c2.iloc[80:100],c3.iloc[80:100],c4.iloc[80:100],c5.iloc[80:100]],ignore_index=True )
    
    train2= pd.concat([c1.iloc[20:],c2.iloc[20:],c3.iloc[20:],c4.iloc[20:],c5.iloc[20:]],ignore_index=True )
    test2 = pd.concat([c1.iloc[:20],c2.iloc[:20],c3.iloc[:20],c4.iloc[:20],c5.iloc[:20]],ignore_index=True )
    
    train3= pd.concat([pd.concat([c1.iloc[:20],c1.iloc[40:]]),pd.concat([c2.iloc[:20],c2.iloc[40:]]),pd.concat([c3.iloc[:20],c3.iloc[40:]]),pd.concat([c4.iloc[:20],c4.iloc[40:]]),pd.concat([c5.iloc[:20],c5.iloc[40:]])],ignore_index=True )
    test3 = pd.concat([c1.iloc[20:40],c2.iloc[20:40],c3.iloc[20:40],c4.iloc[20:40],c5.iloc[20:40]],ignore_index=True )

    train4= pd.concat([pd.concat([c1.iloc[:40],c1.iloc[60:]]),pd.concat([c2.iloc[:40],c2.iloc[60:]]),pd.concat([c3.iloc[:40],c3.iloc[60:]]),pd.concat([c4.iloc[:40],c4.iloc[60:]]),pd.concat([c5.iloc[:40],c5.iloc[60:]])],ignore_index=True )
    test4 = pd.concat([c1.iloc[40:60],c2.iloc[40:60],c3.iloc[40:60],c4.iloc[40:60],c5.iloc[40:60]],ignore_index=True )
    
    train5= pd.concat([pd.concat([c1.iloc[:60],c1.iloc[80:]]),pd.concat([c2.iloc[:60],c2.iloc[80:]]),pd.concat([c3.iloc[:60],c3.iloc[80:]]),pd.concat([c4.iloc[:60],c4.iloc[80:]]),pd.concat([c5.iloc[:60],c5.iloc[80:]])],ignore_index=True )
    test5 = pd.concat([c1.iloc[60:80],c2.iloc[60:80],c3.iloc[60:80],c4.iloc[60:80],c5.iloc[60:80]],ignore_index=True )
    
    return train1,test1,train2,test2,train3,test3,train4,test4,train5,test5
    

fonction d'évaluation des performances

In [173]:
def evaluation(noyau,h,train,test):
    Parzen=apply_noyau(noyau,h,train,test)
    NOYAU=density(Parzen,train,test)
    indice_deux_proches=get_closest(NOYAU,test)
    Matrix,top1,top2=confusion_matrix(test.Classe,indice_deux_proches+1)
    return Matrix,top1,top2

On va chercher le meilleur h parmi les 5 suivantes {1/2,1,3/2,2,5/2}

### En utilisant les données de tp1

In [212]:
train1,test1,train2,test2,train3,test3,train4,test4,train5,test5=cross_validation(data_train)

### Noyau uniform

### h=1/2

In [175]:
h=1/2

In [218]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train1,test1)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 1.0,
 1.0)

In [214]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train2,test2)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 19.,  1.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

In [215]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train3,test3)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 19.,  0.,  1.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

In [216]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train4,test4)
Matrix,top1,top2

(array([[19.,  1.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

In [217]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train5,test5)
Matrix,top1,top2

(array([[19.,  0.,  0.,  0.,  1.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

### h=1

In [219]:
h=1

In [220]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train1,test1)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 1.,  0.,  0., 19.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 0.99)

In [221]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train2,test2)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 1., 19.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 1.,  0.,  0., 19.,  0.],
        [ 0.,  1.,  0.,  0., 19.]]),
 0.97,
 0.99)

In [223]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train3,test3)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 1.,  0., 19.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 0.99)

In [224]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train4,test4)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 2., 18.,  0.,  0.,  0.],
        [ 1.,  0., 19.,  0.,  0.],
        [ 1.,  0.,  0., 19.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.96,
 0.98)

In [225]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train5,test5)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 1.,  0.,  0., 19.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 0.99)

### h=3/2

In [193]:
h=3/2

In [195]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train1,test1)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 1.0,
 1.0)

In [226]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train2,test2)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 1., 19.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 1.,  0.,  0., 19.,  0.],
        [ 0.,  1.,  0.,  0., 19.]]),
 0.97,
 0.99)

In [227]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train3,test3)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 1.,  0., 19.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 0.99)

In [228]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train4,test4)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 2., 18.,  0.,  0.,  0.],
        [ 1.,  0., 19.,  0.,  0.],
        [ 1.,  0.,  0., 19.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.96,
 0.98)

In [229]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train5,test5)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 1.,  0.,  0., 19.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 0.99)

### h=2

In [230]:
h=2

In [231]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train1,test1)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 1.0,
 1.0)

In [232]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train2,test2)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 19.,  1.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

In [233]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train3,test3)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 1.0,
 1.0)

In [234]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train4,test4)
Matrix,top1,top2

(array([[19.,  1.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

In [235]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train5,test5)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  1.,  0.,  0., 19.]]),
 0.99,
 1.0)

### h=5/2

In [236]:
h=5/2

In [202]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train1,test1)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 1.0,
 1.0)

In [237]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train2,test2)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 19.,  1.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

In [238]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train3,test3)
Matrix,top1,top2

(array([[20.,  0.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 19.,  0.,  1.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

In [239]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train4,test4)
Matrix,top1,top2

(array([[19.,  1.,  0.,  0.,  0.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

In [240]:
Matrix,top1,top2=evaluation(noyau_uniform,h,train5,test5)
Matrix,top1,top2

(array([[19.,  0.,  0.,  0.,  1.],
        [ 0., 20.,  0.,  0.,  0.],
        [ 0.,  0., 20.,  0.,  0.],
        [ 0.,  0.,  0., 20.,  0.],
        [ 0.,  0.,  0.,  0., 20.]]),
 0.99,
 1.0)

### Conclusion pour le noyau uniform

Pour le noyau uniform , et comme le montre les résultats ci dessus , la meilleur valeur de h est 2.

Avec h=2 , on a eu 100% comme taux de classification  en utilisant le top2 , et 99.94%  en utilisant le top1

### Noyau gaussien

### h=1/2

In [ ]:
h=1/2

In [242]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train1,test1)
top1,top2

(1.0, 1.0)

In [243]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train2,test2)
top1,top2

(0.99, 1.0)

In [244]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train3,test3)
top1,top2

(1.0, 1.0)

In [245]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train4,test4)
top1,top2

(0.99, 1.0)

In [246]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train5,test5)
top1,top2

(1.0, 1.0)

### h=1

In [251]:
h=1

In [248]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train1,test1)
top1,top2

(0.99, 1.0)

In [249]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train2,test2)
top1,top2

(1.0, 1.0)

In [250]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train3,test3)
top1,top2

(1.0, 1.0)

In [252]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train4,test4)
top1,top2

(1.0, 1.0)

In [253]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train5,test5)
top1,top2

(1.0, 1.0)

### h=2

In [254]:
h=2

In [255]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train1,test1)
top1,top2

(1.0, 1.0)

In [256]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train2,test2)
top1,top2

(1.0, 1.0)

In [257]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train3,test3)
top1,top2

(1.0, 1.0)

In [258]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train4,test4)
top1,top2

(0.99, 1.0)

In [259]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train5,test5)
top1,top2

(1.0, 1.0)

### h=5/2

In [260]:
h=5/2

In [261]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train1,test1)
top1,top2

(1.0, 1.0)

In [262]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train4,test4)
top1,top2

(0.99, 1.0)

In [ ]:
Matrix,top1,top2=evaluation(noyau_gaussien,h,train5,test5)
top1,top2

### Conclusion pour le noyau gaussien

On remarque qu'avec le noyau gaussien , pour different valeur de h , on a eu 100% comme taux de classification en utilisant top2 , et 99.98 % en utilisant top1 . Ca justifie aussi que nos données en plus une forme gaussien qu'uniform